In [14]:
import pandas as pd
import numpy as np

from arch import arch_model
from statsmodels.tsa.arima.model import ARIMA

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
data = pd.read_csv('vodafone_data.csv')

In [3]:
data.head()

,Date,Open,High,Low,Close,Volume
0,2016-02-11,29.330000,29.510000,29.030001,29.270000,4805000
1,2016-02-12,29.520000,29.950001,29.410000,29.900000,3116500
2,2016-02-16,29.950001,30.420000,29.900000,30.260000,4037900
3,2016-02-17,31.000000,31.320000,30.900000,31.090000,4313800
4,2016-02-18,31.090000,31.150000,30.920000,30.959999,3522800


## Підготовка даних

In [4]:
def dataset_preprocess(df):
    df['Date'] = pd.to_datetime(df['Date'])
    df['Date_index'] = pd.DatetimeIndex(df['Date']) 
    
    data_no_missing = df.copy(deep=True)

    data_no_missing = data_no_missing.set_index('Date_index').asfreq('D')

    for col in ['Open', 'High', 'Low', 'Close', 'Volume']:
        data_no_missing[col] = data_no_missing[col].interpolate()

    data_no_missing = data_no_missing.reset_index(drop=False)

    data_no_missing['Date'] = pd.to_datetime(data_no_missing['Date_index']).dt.date
    data_no_missing['year'] = pd.to_datetime(data_no_missing['Date']).dt.year
    data_no_missing['quarter'] = pd.to_datetime(data_no_missing['Date']).dt.quarter
    data_no_missing['month'] = pd.to_datetime(data_no_missing['Date']).dt.month
    
    return df, data_no_missing

In [5]:
df, data_no_missing = dataset_preprocess(data)

### Прогноз ціни на акції на кінець дня на один крок вперед

In [6]:
data_no_missing.head()

,Date_index,Date,Open,High,Low,Close,Volume,year,quarter,month
0,2016-02-11,2016-02-11,29.330000,29.510000,29.030001,29.27,4805000.0,2016,1,2
1,2016-02-12,2016-02-12,29.520000,29.950001,29.410000,29.90,3116500.0,2016,1,2
2,2016-02-13,2016-02-13,29.627500,30.067501,29.532500,29.99,3346850.0,2016,1,2
3,2016-02-14,2016-02-14,29.735000,30.185001,29.655000,30.08,3577200.0,2016,1,2
4,2016-02-15,2016-02-15,29.842501,30.302500,29.777500,30.17,3807550.0,2016,1,2


In [12]:
REAL_CLOSE_NEXT_1_STEP = 18.70

In [8]:
model = ARIMA(data_no_missing['Close'], order=(8, 2, 9))

In [9]:
model_fitted = model.fit()

In [13]:
print(f'Predicted Value: {model_fitted.forecast().values[0]}, Real value: {REAL_CLOSE_NEXT_1_STEP}')

Predicted Value: 18.76108362000851, Real value: 18.7


### Прогноз наступного значення дисперсії на 1 крок

In [15]:
CLOSING_VALUES = data_no_missing.Close.tolist() + [REAL_CLOSE_NEXT_1_STEP]

In [16]:
REAL_VARIANCE_NON_STATIONARY = pd.Series(CLOSING_VALUES).std() ** 2
REAL_VARIANCE_NON_STATIONARY

35.8778397721716

In [17]:
def to_stationary(df, target_column='Close'):
    ts_log = np.log(df[target_column])
    ts_diff = ts_log.diff(periods=1).dropna()
    stationary_ts = ts_diff.diff(periods=1).dropna()
    
    return stationary_ts

In [18]:
df_stationary = to_stationary(data_no_missing)

In [19]:
REAL_VARIANCE_STATIONARY = pd.Series(df_stationary).std() ** 2
REAL_VARIANCE_STATIONARY

0.0003568731924588563

In [20]:
model_arhc = arch_model(np.power(df_stationary, 2), p=17, q=6, vol='GARCH')

In [21]:
model_arhc_fitted = model_arhc.fit()

Iteration:      1,   Func. Count:     27,   Neg. LLF: 4989613007135304.0
Iteration:      2,   Func. Count:     62,   Neg. LLF: -8551.299306739225
Inequality constraints incompatible    (Exit mode 4)
            Current function value: -8551.299297552061
            Iterations: 2
            Function evaluations: 62
            Gradient evaluations: 2


/home/yaroslav/.local/lib/python3.9/site-packages/arch/univariate/base.py:750: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(


In [22]:
model_arhc_fitted.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                  Close   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:                8551.30
Distribution:                  Normal   AIC:                          -17052.6
Method:            Maximum Likelihood   BIC:                          -16914.9
                                        No. Observations:                 1826
Date:                Sun, Mar 21 2021   Df Residuals:                     1825
Time:                        23:15:27   Df Model:                            1
                                   Mean Model                                  
===============================================================================
                  coef    std err          t      P>|t|        95.0% Conf. Int.
-------------------------------------------------------------------------------
mu         -1.6094e-03  1.270e-10 -1.268e+07      0.000 [-1.609e-03,-1.609e-03]
                               Volatility Model                              
=============================================================================
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-----------------------------------------------------------------------------
omega      4.9156e-08  8.046e-14  6.109e+05      0.000  [4.916e-08,4.916e-08]
alpha[1]       0.0118  1.229e-02      0.957      0.338 [-1.232e-02,3.585e-02]
alpha[2]       0.0118  3.380e-02      0.348      0.728 [-5.447e-02,7.800e-02]
alpha[3]       0.0118  4.414e-02      0.267      0.790 [-7.474e-02,9.827e-02]
alpha[4]       0.0118  6.637e-02      0.177      0.859      [ -0.118,  0.142]
alpha[5]       0.0118  5.328e-02      0.221      0.825   [-9.266e-02,  0.116]
alpha[6]       0.0118  6.903e-02      0.170      0.865      [ -0.124,  0.147]
alpha[7]       0.0118  5.448e-02      0.216      0.829   [-9.501e-02,  0.119]
alpha[8]       0.0118  3.054e-02      0.385      0.700 [-4.809e-02,7.162e-02]
alpha[9]       0.0118  5.103e-02      0.231      0.818   [-8.826e-02,  0.112]
alpha[10]      0.0118  8.077e-02      0.146      0.884      [ -0.147,  0.170]
alpha[11]      0.0118      0.138  8.530e-02      0.932      [ -0.259,  0.282]
alpha[12]      0.0118      0.130  9.025e-02      0.928      [ -0.244,  0.267]
alpha[13]      0.0118      0.132  8.940e-02      0.929      [ -0.246,  0.270]
alpha[14]      0.0118      0.109      0.108      0.914      [ -0.201,  0.225]
alpha[15]      0.0118      0.484  2.428e-02      0.981      [ -0.938,  0.961]
alpha[16]      0.0118      0.311  3.783e-02      0.970      [ -0.598,  0.621]
alpha[17]      0.0118      0.599  1.963e-02      0.984      [ -1.163,  1.187]
beta[1]        0.1300      0.174      0.747      0.455      [ -0.211,  0.471]
beta[2]        0.1300      0.165      0.786      0.432      [ -0.194,  0.454]
beta[3]        0.1300      0.198      0.655      0.512      [ -0.259,  0.519]
beta[4]        0.1300      0.463      0.281      0.779      [ -0.778,  1.038]
beta[5]        0.1300      0.332      0.392      0.695      [ -0.520,  0.780]
beta[6]        0.1300      0.652      0.199      0.842      [ -1.148,  1.408]
=============================================================================

Covariance estimator: robust
WARNING: The optimizer did not indicate successful convergence. The message was Inequality constraints incompatible.
See convergence_flag.

"""

In [25]:
print(f"""Predicted Value: {model_arhc_fitted.forecast(horizon=1).variance.values[-1][0]},
      Real Value: {REAL_VARIANCE_STATIONARY}""")

Predicted Value: 5.168567792006029e-06,
      Real Value: 0.0003568731924588563
